In [1]:
import os, sys
sys.path.insert(0, '/data/yosef2/users/can/PopV/PopV')
import popv
import anndata
import numpy as np
import scanpy as sc
import scvi
import gc
import os

2023-01-30 14:38:34.989326: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-30 14:38:37.173435: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-30 14:38:37.173613: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-30 14:38:37.173631: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

Instructions for updating:
non-resource variables are not supported in the long term


Global seed set to 0


In [13]:
import requests
res = requests.get("https://zenodo.org/api/records/7587774")
tissue_download_path = {ind['key'][3:-14]:ind['links']['self'] for ind in res.json()['files']}

In [4]:
input_file ={'source': 'wget', 'path': 'downloaded_ts_data_popv/LCA.h5ad', 'link': 'https://www.dropbox.com/s/mrf8y7emfupo4he/LCA.h5ad?dl=1'}

if input_file['source']== 'gdrive':
  # OPTION 1: Connect to Google Drive
  # This is the recomended method especially for large datasets
  from google.colab import drive    
  drive.mount('/content/drive')
  query_adata = anndata.read(input_file['path'])
elif input_file['source'] == 'local':
  # OPTION 2: Uploading data manually
  # Click the folder icon on the left navigation bar, and select the upload icon
  # Note: Manually uploaded data is automatically deleted when the colab session ends
  # This is not recommended if your dataset is very large
  query_adata = anndata.read(input_file['path'])
else:
  # OPTION 3: Downloading from the cloud (Dropbox, AWS, Google Drive, etc)
  # Google Colab supports wget, curl, and gdown commands
  # It is recommended to download the data into Google Drive and read from there.
  # This way your data will be persistent.
  print('downloading')
  try:
    !wget -O {input_file['path']} {input_file['link']}
    query_adata = anndata.read(input_file['path'])
  except:
    raise Exception(f'Default download failed with wget. Use custom downloader or check provided link ' + input_file['link'])
    
query_adata.obs_names_make_unique()
query_adata.var_names = query_adata.var_names.str.upper()

downloading
--2023-01-30 14:38:53--  https://www.dropbox.com/s/mrf8y7emfupo4he/LCA.h5ad?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/mrf8y7emfupo4he/LCA.h5ad [following]
--2023-01-30 14:38:53--  https://www.dropbox.com/s/dl/mrf8y7emfupo4he/LCA.h5ad
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd8169d1d68f4fe72546f0366d1.dl.dropboxusercontent.com/cd/0/get/B1kqB2pTz7_2-w1FXiUmM2JHNysH2tqDypjRRmiIEEF--A2ATu9cfmXSZwnwewtY86tIPDBFgH66TBW9WhWUHFkSVAB6RnJfVUBKE7FFACkbZI7XdbXo19m9k_nnGwgHo5EM92LZMKBxnALwW8HC8xkGDLbBQF5I-kudt_dKaX5j-NWKFYGwtq8DC09w_vY5Z0g/file?dl=1# [following]
--2023-01-30 14:38:53--  https://ucd8169d1d68f4fe72546f0366d1.dl.dropboxusercontent.com/cd/0/get/B1kqB2pTz7_2-w1FXiUmM2JHNysH2tqDypjRRmiIEEF--A2ATu9cfmXSZ

In [5]:
query_adata = sc.read('downloaded_ts_data_popv/LCA.h5ad')
query_adata = query_adata[query_adata.obs_names[0:5]].copy()

In [6]:
query_adata

AnnData object with n_obs × n_vars = 5 × 23681
    obs: 'method', 'donor', 'cell_ontology_type', 'donor_method', 'cell_ontology_id'

In [7]:
# Following parameters are specific to Tabula Sapiens dataset
ref_labels_key='cell_ontology_class'
ref_batch_key = 'donor_assay'

query_batch_key = 'donor_method'
query_labels_key=None
unknown_celltype_label='unknown'

# Folder structure for files and models.
data_dir = 'downloaded_ts_data_popv/'
model_dir = 'pretrained_predictors/'

In [8]:
from popv.preprocessing import Process_Query
from popv.annotation import annotate_data

In [9]:
sc.settings.figdir = model_dir

In [11]:
2

2

In [ ]:
""" 
tissue options: 

"""
# Done 
#  - Following label not in celltype_dict  intestinal transient amplifying cell
# Lung - No path between capillary aerocyte and cell.

finished = []

for tissue in ['Salivary_Gland', 'Small_Intestine']: #tissue_download_path.keys()::
    if tissue in finished:
        print(f'{tissue} already finished. Skipping recomputation!')
        continue
    refdata_url = tissue_download_path[tissue]
    print(tissue, refdata_url)
    if refdata_url is None:
        continue
    # Download reference dataset
    output_fn = data_dir + f'TS_{tissue}.h5ad'
    !wget -O $output_fn $refdata_url
    ref_adata = anndata.read(output_fn)

    min_celltype_size = np.min(ref_adata.obs.groupby(ref_labels_key).size())
    n_samples_per_label = np.max((min_celltype_size, 500))

    adata = Process_Query(
            query_adata,
            ref_adata,
            query_labels_key=query_labels_key,
            query_batch_key=query_batch_key,
            ref_labels_key=ref_labels_key,
            ref_batch_key=ref_batch_key,
            
            unknown_celltype_label=unknown_celltype_label,
            save_path_trained_models=model_dir+tissue+'_pretrained/',
            prediction_mode='retrain', # 'fast' mode gives fast results (does not include BBKNN and Scanorama and makes more inaccurate errors)
            n_samples_per_label=n_samples_per_label,
            use_gpu=2,
            compute_embedding=True,
            hvg=4000
        ).adata
    annotate_data(adata, save_path=None)
    adata.obsm['X_umap'] = adata.obsm['X_scanvi_umap_popv']
    _ = sc.pl.umap(adata, color=[ref_labels_key, ref_batch_key], ncols=1, save=tissue+'.pdf', frameon=False, show=False)
    gc.collect()

Salivary_Gland https://zenodo.org/api/files/0dfd03a7-d45c-43cd-b79a-779bbac161df/TS_Salivary_Gland_filtered.h5ad
--2023-01-30 23:40:38--  https://zenodo.org/api/files/0dfd03a7-d45c-43cd-b79a-779bbac161df/TS_Salivary_Gland_filtered.h5ad
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1142910098 (1.1G) [application/octet-stream]
Saving to: ‘downloaded_ts_data_popv/TS_Salivary_Gland.h5ad’

downloaded_ts_data_   4%[                    ]  47.15M  22.2KB/s    in 8m 55s  

2023-01-30 23:49:34 (90.3 KB/s) - Read error at byte 49445860/1142910098 (Connection reset by peer). Retrying.

--2023-01-30 23:49:35--  (try: 2)  https://zenodo.org/api/files/0dfd03a7-d45c-43cd-b79a-779bbac161df/TS_Salivary_Gland_filtered.h5ad
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 1142910098 (1

Sampling 500 per label
Saving celltypist results to adata.obs["popv_celltypist_prediction"]
🍳 Preparing data before training
⚖️ Scaling input data
🏋️ Training data using logistic regression
✅ Model training done!
🔬 Input data has 27199 cells and 4000 genes
🔗 Matching reference genes in the model
🧬 4000 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!
👀 Can not detect a neighborhood graph, will construct one before the over-clustering
⛓️ Over-clustering input data with resolution set to 15
🗳️ Majority voting the predictions
✅ Majority voting done!
Integrating data with bbknn
Saving knn on bbknn results to adata.obs["popv_knn_on_bbknn_prediction"]
Saving UMAP of bbknn results to adata.obs["X_bbknn_umap_popv"]
Integrating data with scanorama


Found 4000 genes among all datasets
[[0.         0.24067403 0.01081731 0.05438993]
 [0.         0.         0.50721154 0.15187218]
 [0.         0.         0.         0.37019231]
 [0.         0.         0.         0.        ]]
Processing datasets (1, 2)
Processing datasets (2, 3)
Processing datasets (0, 1)
Processing datasets (1, 3)


Saving knn on scanorama results to adata.obs["popv_knn_on_scanorama_prediction"]
Saving UMAP of scanorama results to adata.obs["X_scanorama_umap_popv"]
Integrating data with scvi
Training scvi offline.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Epoch 147/147: 100%|██████████████████████████████████████████| 147/147 [10:12<00:00,  5.12s/it, loss=1.62e+03, v_num=1]

...

Epoch 20/20: 100%|██████████████████████████████████████████████| 20/20 [00:57<00:00,  2.88s/it, loss=1.75e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 20/20: 100%|██████████████████████████████████████████████| 20/20 [00:57<00:00,  2.88s/it, loss=1.75e+03, v_num=1]


Saving scanvi label prediction to adata.obs["popv_scanvi_prediction"]
Saving UMAP of scanvi results to adata.obs["X_scanvi_umap_popv"]
Computing support vector machine. Storing prediction in adata.obs["popv_svm_prediction"]
Using predictions ['popv_celltypist_prediction', 'popv_knn_on_bbknn_prediction', 'popv_knn_on_scanorama_prediction', 'popv_knn_on_scvi_prediction', 'popv_onclass_prediction', 'popv_rf_prediction', 'popv_scanvi_prediction', 'popv_svm_prediction'] for PopV consensus
... storing '_batch_annotation' as categorical
... storing '_labels_annotation' as categorical
... storing 'popv_knn_on_bbknn_prediction' as categorical
... storing 'popv_knn_on_scanorama_prediction' as categorical
... storing 'popv_knn_on_scvi_prediction' as categorical
... storing '_labels_annotation_cell_ontology_id' as categorical
... storing 'popv_onclass_prediction' as categorical
... storing 'onclass_seen' as categorical
... storing 'popv_rf_prediction' as categorical
... storing 'popv_scanvi_predic

Small_Intestine https://zenodo.org/api/files/0dfd03a7-d45c-43cd-b79a-779bbac161df/TS_Small_Intestine_filtered.h5ad
--2023-01-31 00:46:55--  https://zenodo.org/api/files/0dfd03a7-d45c-43cd-b79a-779bbac161df/TS_Small_Intestine_filtered.h5ad
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 345399676 (329M) [application/octet-stream]
Saving to: ‘downloaded_ts_data_popv/TS_Small_Intestine.h5ad’

                dow  95%[==================> ] 315.19M  1.29MB/s    eta 19s    

In [ ]:
import scvi
import gc
import scanpy as sc

In [ ]:
tissues = ['Blood', 'Bone_Marrow', 'Fat', 'Heart', 'Kidney', 'Large_Intestine', 'Liver', 'Lung', 'Lymph_Node', 'Mammary', 'Muscle', 'Pancreas',
           'Prostate', 'Salivary_Gland', 'Skin', 'Small_Intestine', 'Spleen', 'Thymus', 'Trachea', 'Vasculature']

""" 
tissue options: 

"""

batch_size = 256

# Done 
#  - Following label not in celltype_dict  intestinal transient amplifying cell
# Lung - No path between capillary aerocyte and cell.

for tissue in tissues:
    adata = sc.read(f"pretrained_model_all_latest/trained_models_{tissue}/adata.h5ad")
    scvi.model.CondSCVI.setup_anndata(
        adata,
        labels_key="_labels_annotation",
        layer="scvi_counts",
    )
    
    condscvi_model = scvi.model.CondSCVI(
        adata,
        n_latent=5,
        n_layers=2,
        dropout_rate=0.05,
        weight_obs=False
    )

    condscvi_model.train(max_epochs=200, train_size=1.0, batch_size=batch_size)
    condscvi_model.save(f"pretrained_model_all_latest/trained_models_{tissue}/condscvi", overwrite=True, save_anndata=False)
    
    scvi.external.RNAStereoscope.setup_anndata(
        adata,
        labels_key = "_labels_annotation",
        layer = "scvi_counts")
    stereoscope_model = scvi.external.stereoscope.RNAStereoscope(
        adata
    )
    stereoscope_model.train()
    stereoscope_model.save(f"pretrained_model_all_latest/trained_models_{tissue}/rna_stereoscope", overwrite=True, save_anndata=False)
    
    print('Finished ', tissue)
    gc.collect()

In [ ]:
import tarfile

In [ ]:
for tissue in tissues:
    !tar -czvf pretrained_model_all_other_filtering/{tissue}_pretrained_ts.tar.gz -C pretrained_model_all_other_filtering/trained_models_{tissue}/ .
    #!rm -rf pretrained_model_all_other_filtering/trained_models_{tissue}
    !file {tissue}_pretrained_ts.tar.gz

In [ ]:
def compress(tissue, output_dir='', root_dir='.'):
    """compress dirs.

    KWArgs
    ------
    output_file : str, default ="archive.tar.gz"
    output_dir : str, default = ''
        absolute path to output
    root_dir='.',
        absolute path to input root dir
    items : list
        list of dirs/items relative to root dir

    """
    items = ["pretrained_model_all/trained_models_" + tissue]
    print(items)
    with tarfile.open("pretrained_model_all/" + tissue + '_pretrained_ts.tar.gz', "w") as tar:
        for item in items:
            print(item)
            tar.add(item, arcname=item)

for tissue in tissues:
    print(tissue)
    compress(tissue=tissue)